In [1]:
import pandas as pd
import os
tama_df = pd.read_csv('/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/data/geocode_tama_address_data.csv')
tama_df.columns

Index(['Unnamed: 0', 'MezheYeshut', 'GISId', 'MisparYeshut', 'MisparYeshutAv',
       'AchuzHachtamot', 'AchuzHachtamotMitcham', 'Bayt', 'GushChelka',
       'KamutDirot', 'KamutTziburi', 'kaymetminhelet', 'KodMinhelet',
       'KtovetAishKesher', 'MachozMinhal', 'MahutBakasha', 'MechirMemutza',
       'mitve', 'Rechov', 'SemelIishuv', 'ShemMitcham', 'ShnatBakasha',
       'ShnatHryter', 'ShnatIchlus', 'Status', 'TaarichMechiraRishona',
       'YachadKayam', 'YachadMutza', 'YachadTabo', 'YachadTosafti', 'Yazam',
       'Yeshut', 'Yeshuv', 'BakashaLeHeter', 'Heter1', 'Ichlus1', 'address',
       'coordinates', 'latitude', 'longitude'],
      dtype='object')

### engineer total_open_requests feature

In [2]:
years_list, cities_list = list(range(2005, 2023)), list(set(tama_df['Yeshuv']))
years_num, cities_num = len(years_list), len(cities_list)
list_of_cities_list = [[city] * years_num for city in cities_list]
cities = []
for city in list_of_cities_list:
    cities.extend(city)
total_open_requests_per_city_year = pd.DataFrame(data={'year': years_list * cities_num, 
                                                       'Yeshuv': cities
                                                      })
total_open_requests_per_city_year.sort_values(by=['Yeshuv', 'year'], ignore_index=True, inplace=True)
total_open_requests_per_city_year

,year,Yeshuv
0,2005,אור יהודה
1,2006,אור יהודה
2,2007,אור יהודה
3,2008,אור יהודה
4,2009,אור יהודה
...,...,...
1075,2018,תל אביב יפו
1076,2019,תל אביב יפו
1077,2020,תל אביב יפו
1078,2021,תל אביב יפו


In [3]:
def get_total_open_requests(row, tama_df):
    current_city_open_requests_df = tama_df[(tama_df['Yeshuv'] == row['Yeshuv']) & 
                                            (tama_df['ShnatBakasha'] < row['year']) &
                                            ((tama_df['ShnatHryter'] >= row['year']) | (pd.isna(tama_df['ShnatHryter'])))
                                           ]
    return current_city_open_requests_df.shape[0]

In [4]:
total_open_requests_per_city_year['total_open_requests'] = total_open_requests_per_city_year.apply(
    get_total_open_requests, args=(tama_df, ), axis=1)
total_open_requests_per_city_year.sort_values(by=['Yeshuv', 'year'], ignore_index=True, inplace=True)
total_open_requests_per_city_year

,year,Yeshuv,total_open_requests
0,2005,אור יהודה,0
1,2006,אור יהודה,0
2,2007,אור יהודה,0
3,2008,אור יהודה,0
4,2009,אור יהודה,1
...,...,...,...
1075,2018,תל אביב יפו,687
1076,2019,תל אביב יפו,768
1077,2020,תל אביב יפו,735
1078,2021,תל אביב יפו,858


In [5]:
semel_yeshuv_map = tama_df[['SemelIishuv', 'Yeshuv']].drop_duplicates().sort_values(by=['SemelIishuv', 'Yeshuv'])

In [6]:
total_open_requests_per_city_year = total_open_requests_per_city_year.merge(
    semel_yeshuv_map, on=['Yeshuv']).sort_values(by=['Yeshuv', 'year'], ignore_index=True)
total_open_requests_per_city_year

,year,Yeshuv,total_open_requests,SemelIishuv
0,2005,אור יהודה,0,2400
1,2006,אור יהודה,0,2400
2,2007,אור יהודה,0,2400
3,2008,אור יהודה,0,2400
4,2009,אור יהודה,1,2400
...,...,...,...,...
1075,2018,תל אביב יפו,687,5000
1076,2019,תל אביב יפו,768,5000
1077,2020,תל אביב יפו,735,5000
1078,2021,תל אביב יפו,858,5000


In [7]:
total_requests_per_city_and_year_df = tama_df.groupby(by=['SemelIishuv', 'Yeshuv', 'ShnatBakasha'], as_index=False).size()
total_requests_per_city_and_year_df.rename(
    columns={'ShnatBakasha': 'year', 'size': 'total_requests'}, inplace=True)


total_approved_requests_per_city_and_year_df = tama_df.groupby(by=['SemelIishuv', 'Yeshuv', 'ShnatHryter'], as_index=False).size()
total_approved_requests_per_city_and_year_df.rename(
    columns={'ShnatHryter': 'year', 'size': 'total_approved_requests'}, inplace=True)

total_open_requests_per_city_year = total_open_requests_per_city_year.merge(
    total_requests_per_city_and_year_df, on=['SemelIishuv', 'Yeshuv', 'year'], how='left').merge(
    total_approved_requests_per_city_and_year_df, on=['SemelIishuv', 'Yeshuv', 'year'], how='left')

total_open_requests_per_city_year = total_open_requests_per_city_year.fillna(0)
total_open_requests_per_city_year.sort_values(by=['Yeshuv', 'year'], ignore_index=True, inplace=True)
total_open_requests_per_city_year

,year,Yeshuv,total_open_requests,SemelIishuv,total_requests,total_approved_requests
0,2005,אור יהודה,0,2400,0.0,0.0
1,2006,אור יהודה,0,2400,0.0,0.0
2,2007,אור יהודה,0,2400,0.0,0.0
3,2008,אור יהודה,0,2400,1.0,0.0
4,2009,אור יהודה,1,2400,0.0,0.0
...,...,...,...,...,...,...
1075,2018,תל אביב יפו,687,5000,164.0,83.0
1076,2019,תל אביב יפו,768,5000,116.0,149.0
1077,2020,תל אביב יפו,735,5000,238.0,115.0
1078,2021,תל אביב יפו,858,5000,240.0,124.0


In [8]:
cols = ['Yeshuv', 'SemelIishuv', 'year', 'total_requests', 'total_approved_requests', 'total_open_requests']
total_open_requests_per_city_year = total_open_requests_per_city_year[cols]
total_open_requests_per_city_year.sort_values(by=cols, ignore_index=True, inplace=True)
total_open_requests_per_city_year.to_csv('/Users/sararozin/Documents/Tama/Tama/total_approved_request_per_year/data/total_requests_per_city_and_year_v3.csv', index=False)
total_open_requests_per_city_year

,Yeshuv,SemelIishuv,year,total_requests,total_approved_requests,total_open_requests
0,אור יהודה,2400,2005,0.0,0.0,0
1,אור יהודה,2400,2006,0.0,0.0,0
2,אור יהודה,2400,2007,0.0,0.0,0
3,אור יהודה,2400,2008,1.0,0.0,0
4,אור יהודה,2400,2009,0.0,0.0,1
...,...,...,...,...,...,...
1075,תל אביב יפו,5000,2018,164.0,83.0,687
1076,תל אביב יפו,5000,2019,116.0,149.0,768
1077,תל אביב יפו,5000,2020,238.0,115.0,735
1078,תל אביב יפו,5000,2021,240.0,124.0,858


### frontend code for new year prediction

In [9]:
def calculate_total_open_request(city, futured_year, tama_df):
    current_city_open_requests_df = tama_df[
        (tama_df['Yeshuv'] == city) & 
        (tama_df['ShnatBakasha'] < futured_year) & 
        ((tama_df['ShnatHryter'] >= futured_year) | (pd.isna(tama_df['ShnatHryter'])))
    ]
    return current_city_open_requests_df.shape[0]

In [10]:
calculate_total_open_request(2003, 2400, tama_df)

0